In [1]:
import requests
import json
import pandas as pd 
import sqlite3

conn = sqlite3.connect('spotify2.db')
c = conn.cursor()

In [43]:
headers_value = 'BQCPKnZgUWqLakMHo1WSLy-V3NoGZQGBZbGenL_obxJY-Qe39C7n5r-SCuUvjyPDlrrjQJ_d9TvNlBE4CXkm_mFCy2UxOSimWm6SuoY-dDM6VGeseh8Y0wJdJ0w0of75Au_FoKlhGNpg9gwEhZo'
headers = {'Authorization':f'Bearer {headers_value}'}
        
def get_album_SQL(album_id, table_name):
    resp = requests.get(f'https://api.spotify.com/v1/albums/{album_id}/tracks', headers = headers)
    data_json = resp.json()
    
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name}
                 ([index] INTEGER PRIMARY KEY,
                 [song_name] text, 
                 [song_id] text,
                 [album_name] text,
                 [popularity] integer,
                 [artist_name] text,
                 [speechiness] float, 
                 [danceability] float)''')

        
    for i in data_json['items']:  
        c.execute(f'''INSERT INTO {table_name} 
                    (song_name, song_id, album_name)
                    VALUES(?,?)''', 
                    (i['name'],
                     i['id'])
                     )
    
        conn.commit()
         
        c.execute(f'SELECT * FROM {table_name}')
        
        
        c.fetchall()

        

In [73]:
resp = requests.get(f'https://api.spotify.com/v1/albums/1LShhEEKRT5MNPcO7jtYHh/tracks', headers = headers)
data_json = resp.json()

In [77]:
data_json.total

AttributeError: 'dict' object has no attribute 'total'

In [55]:
import math
def song_audio_features(df, table_name):
    '''takes in dataframe table and corresponding SQL table and returns dataframe with audio features for the songs'''
   
    song_ids = list(df['song_id'])
    
    
    #if len(song_ids) < 100:
    for i in range(0, math.ceil(len(song_ids)/100)):
        abbr_list = song_ids[i:i+100]    
        
        formatted = (",".join(abbr_list))
        resp = requests.get(f'https://api.spotify.com/v1/audio-features?ids={formatted}', headers = headers)
        data_json = resp.json()
        #print(data_json)
        for j in data_json['audio_features']:
            #print(i['speechiness'], i['danceability'], i['id'])
            try:
                c.execute(f'''UPDATE {table_name} 
                              SET speechiness = ?, 
                                  danceability = ?
                              WHERE song_id = ?''', (j['speechiness'], j['danceability'], j['id']))
                conn.commit()
            except:
                print(j)
     
    return SQL_to_dataframe(f'{table_name}')
    
   

In [57]:
drake_df = song_audio_features(df, 'drake_albums')

In [ ]:
get_album_SQL('1LShhEEKRT5MNPcO7jtYHh', 'drake_albums')

In [58]:
get_album_SQL('1ATL5GLyefJaxhQzSPVrLX', 'drake_albums')

In [59]:
get_album_SQL('1lXY618HWkwYKJWBRYR4MK', 'drake_albums')

In [60]:
get_album_SQL('40GMAhriYJRO1rsY4YdrZb', 'drake_albums')

In [61]:
get_album_SQL('1ozpmkWcCHwsQ4QTnxOOdT', 'drake_albums')

In [62]:
get_album_SQL('0ptlfJfwGTy0Yvrk14JK1I', 'drake_albums')

In [63]:
get_album_SQL('2gXTTQ713nCELgPOS0qWyt', 'drake_albums')

In [64]:
get_album_SQL('6X1x82kppWZmDzlXXK3y3q', 'drake_albums')

In [65]:
get_album_SQL('6agmeioaDOBupymziJihgB', 'drake_albums')

In [66]:
df = SQL_to_dataframe('drake_albums')

In [67]:
drake_df = song_audio_features(df, 'drake_albums')

In [70]:
drake_df

,song_name,song_id,popularity,artist_name,speechiness,danceability
index,,,,,,
1,Lust For Life,1v24T2ug4TlssYZvI3aL4O,None,None,0.1750,0.462
2,Houstatlantavegas,3XLSlQLJf3Ut0zvMUxnF1h,None,None,0.0574,0.683
3,Successful (feat. Trey Songz & Lil Wayne),6NwhtCYkfwWlcR62Ac4E7y,None,None,0.3690,0.562
4,Let’s Call It Off (feat. Peter Bjorn and John),4FDSPGncHRpr6yB8DD4CtI,None,None,0.0327,0.790
5,November 18th,0JaVdpmiex2EP7bBzyKVTa,None,None,0.5070,0.749
6,Ignant Shit (feat. Lil Wayne),2hcohLIysMxofYziluXCoX,None,None,0.2760,0.591
7,A Night Off (feat. Lloyd),3lkkkYc1wBj5l3FVlpqvP6,None,None,0.0741,0.611
8,Say What's Real,7mPoCVGP752A5DtHf1SKkM,None,None,0.2880,0.462
9,Little Bit (feat. Lykke Li),524wvipGqxPKYWxkjf9y46,None,None,0.0374,0.758
